# Tâche #2 : Classification d'incidents avec un réseau  récurrent et des *embeddings* Spacy

Cette tâche est similaire à la précédente et vous réutilisez les mêmes fichiers d’entraînement, de validation et de test. Cependant, vous devez utiliser des réseaux récurrents pour classifier les textes. Plus particulièrement, vous devez entraîner un réseau de neurones LSTM pour encoder les textes et une couche linéaire pour faire la classification des textes. 

Les consignes pour cette tâche sont: 
- 	Nom du notebook : rnn.ipynb
- 	Tokenisation : Utilisation de Spacy. 
- 	Plongements de mots : Ceux de Spacy. 
- 	Normalisation : Aucune normalisation. 
- 	Structure du réseau : Un réseau LSTM avec 1 seule couche pour l’encodage de textes. Je vous laisse déterminer la taille de cette couche (à expliquer). 
- 	Analyse : Comparer les résultats obtenus avec un réseau unidirectionnel et un réseau bidirectionnel. Si vous éprouvez des difficultés à entraîner les 2 réseaux dans un même notebook, faites une copie et nommez le 2e fichier rnn-bidirectionnel.ipynb.
- 	Expliquez comment les modèles sont utilisés pour faire la classification d’un texte. 
- 	Présentez clairement vos résultats et faites-en l’analyse. 


Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle. 

## 1. Création du jeu de données (*dataset*)

In [4]:
import spacy

spacy_model = spacy.load("en_core_web_md")
embedding_size = spacy_model.meta['vectors']['width']


import pandas as pd
import json
import numpy as np
# Assurez-vous que le modèle de langue de spacy est téléchargé
# python -m spacy download fr_core_news_md (par exemple pour le français)

# Charger le modèle de langue de spacy

# Définition des chemins vers les fichiers de données
train_data_path = './data/incidents_train.json'
dev_data_path = './data/incidents_dev.json'
test_data_path = './data/incidents_test.json'

def load_incident_dataset(filename):
    with open(filename, 'r') as fp:
        incident_list = json.load(fp)
        
        text = [item["text"] for item in incident_list]
        target = np.array([int(item["label"]) for item in incident_list])
         
    return text, target


# Créer les DataFrames pour chaque partition de données
train_list, train_target = load_incident_dataset(train_data_path)
dev_list, dev_target = load_incident_dataset(dev_data_path)
test_list, test_target = load_incident_dataset(test_data_path)

# Affichage de l'information de base sur les DataFrames
display(f"Train data: text_size {len(train_list)}, target_size {len(train_target)}")
display(f"Dev data: text_size {len(dev_list)}, target_size {len(dev_target)}")
display(f"Test data: text_size {len(test_list)}, target_size {len(test_target)}")



# Vérification des premiers enregistrements dans l'ensemble d'entraînement
train_list[0]


'Train data: text_size 2475, target_size 2475'

'Dev data: text_size 531, target_size 531'

'Test data: text_size 531, target_size 531'

' At approximately 8:50 a.m. on October 29  1997  Employee #1 was painting a  single story house at 2657 7th Ave  Sacramento  CA. He was caulking around the  peak of the roof line on the west side of the house  20 ft above the ground.  He was working off of a 24 ft aluminum extension ladder so that his feet were  approximately 12 to 13 feet above the ground. Employee #1 fell and suffered a  concussion and two dislocated discs in his lower back and was hospitalized.  The ladder was not secured to prevent movement.                                 '

In [5]:
import numpy as np

padding_token = "<PAD>"   # mot 0
unk_token = "<UNK>"    # mot 1
zero_vec_embedding = np.zeros(embedding_size, dtype=np.float64)

id2word = {}
id2word[0] = padding_token 
id2word[1] = unk_token 

word2id = {}
word2id[padding_token] = 0
word2id[unk_token] = 1

id2embedding = {}
id2embedding[0] = zero_vec_embedding
id2embedding[1] = zero_vec_embedding

## 2. Gestion de plongements de mots (*embeddings*)

In [6]:
from torch import FloatTensor

def get_spacy_embeddings(text, spacy_analyzer=spacy_model):
    doc = spacy_analyzer(text)
    embeddings = [token.vector for token in doc]
    return FloatTensor(embeddings)

In [7]:
word_index = 2
vocab = word2id.keys()
for question in train_list:
    for word in spacy_model(question):
        if word.text not in vocab:
            word2id[word.text] = word_index
            id2word[word_index] = word.text
            id2embedding[word_index] = word.vector
            word_index += 1

In [8]:
id2word[4]

'approximately'

In [9]:
from torch.utils.data import Dataset, DataLoader
from torch import FloatTensor, LongTensor, unsqueeze
from typing import List
import numpy as np

class SpacyDataset(Dataset):
    def __init__(self, dataset: List[str] , target: np.array, wordId: dict, model=spacy_model):
        self.dataset = dataset
        self.doc_embeddings = [None for _ in range(len(dataset))]
        self.targets = target
        self.tokenizer = model
        self.word2id = wordId

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        if self.doc_embeddings[index] is None:
            self.doc_embeddings[index] = self.tokenize(self.dataset[index])
        return LongTensor(self.doc_embeddings[index]), LongTensor([self.targets[index]]).squeeze(0)

    def tokenize(self, sentence):
        tokens = [word.text for word in self.tokenizer(sentence)]
        return [self.word2id.get(token, 1) for token in tokens]  # get(token, 1) retourne 1 par défaut si mot inconnu

In [10]:
import torch
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence

def pad_batch(batch):
    x = [x for x,y in batch]
    x_true_length = [len(x) for x,y in batch]
    y = torch.stack([y for x,y in batch], dim=0)
    return ((pad_sequence(x, batch_first=True), x_true_length), y)

In [11]:
# On finalise la construction des 3 jeux de données et leurs dataloaders
train_dataset = SpacyDataset(train_list, train_target, word2id)
valid_dataset = SpacyDataset(dev_list, dev_target, word2id)
test_dataset = SpacyDataset(test_list, test_target, word2id)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True)

In [12]:
data = next(iter(train_dataloader))


## 3. Création de modèle(s)

In [13]:
vocab_size = len(id2embedding)
embedding_layer = np.zeros((vocab_size, embedding_size), dtype=np.float32)
for token_id, embedding in id2embedding.items():
    embedding_layer[token_id,:] = embedding
embedding_layer = torch.from_numpy(embedding_layer)

print("Taille de la couche d'embeddings:", embedding_layer.shape)



Taille de la couche d'embeddings: torch.Size([11642, 300])


In [14]:
# import torch.nn as nn
# from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

# class LSTMClassifier(nn.Module):
#     def __init__(self, embeddings, hidden_state_size, nb_classes) :
#         super(LSTMClassifier, self).__init__()
#         self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
#         self.embedding_size = embeddings.size()[1]
#         self.rnn = nn.LSTM(self.embedding_size, hidden_state_size, 1, batch_first=True)
#         self.classification_layer = nn.Linear(hidden_state_size, nb_classes)

#     def forward(self, x, x_lengths):
#         x = self.embedding_layer(x)
#         packed_batch = pack_padded_sequence(x, x_lengths, batch_first=True, enforce_sorted=False)
#         x, last_hidden_state = self.rnn(packed_batch)  # On utilise le hidden state de la dernière cellule
#         # x = last_hidden_state.squeeze()  # Le LSTM a une seule couche, on retire cette dimension
#         x = self.classification_layer(x)
#         x = nn.functional.softmax(x)
#         return x

In [15]:
import torch.nn as nn
import torch.nn.functional as F

class LSTMClassifier(nn.Module):
    def __init__(self, embeddings, hidden_state_size, nb_classes, bidirectional=False):
        super(LSTMClassifier, self).__init__()
        self.embedding_layer = nn.Embedding.from_pretrained(embeddings)
        self.rnn = nn.LSTM(input_size=embeddings.size()[1], hidden_size=hidden_state_size, batch_first=True, bidirectional=bidirectional)
        multiplier = 2 if bidirectional else 1
        self.classification_layer = nn.Linear(hidden_state_size * multiplier, nb_classes)

    def forward(self, x, x_lenght):
        x = self.embedding_layer(x)
        packed_batch = pack_padded_sequence(x, x_lenght, batch_first=True, enforce_sorted=False)
        packed_output, (hidden_state, cell_state) = self.rnn(packed_batch)

        if self.rnn.bidirectional:
            # Concatenating the hidden states of the last time step from both directions
            x = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim=1)
        else:
            x = hidden_state[-1,:,:]  # Taking the last time step of the hidden state

        x = self.classification_layer(x)
        return x


In [16]:
from poutyne import set_seeds

set_seeds(42)
hidden_size = 100  # choisi arbitrairement
nb_classes = 9
model = LSTMClassifier(embedding_layer, hidden_size, nb_classes)

In [17]:
from poutyne import set_seeds

set_seeds(42)
hidden_size = 100  # choisi arbitrairement
nb_classes = 9
bidirectional_model = LSTMClassifier(embedding_layer, hidden_size, nb_classes, bidirectional=True)

## 4. Fonctions utilitaires

Vous pouvez mettre ici toutes les fonctions qui seront utiles pour les sections suivantes.

## 5. Entraînement de modèle(s)

In [18]:
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np

set_seeds(42)
hidden_size = 100
optimizer = "ADAM"

directory_name = 'model2/_mlp_optimizer{}'.format(optimizer)

experiment = Experiment(directory_name,
                        model,
                        optimizer = optimizer,
                        task="classification",
                        loss_function="cross_entropy",
                        )

In [19]:
logging = experiment.train(train_dataloader, valid_dataloader, epochs=50, disable_tensorboard=True)

Loading weights from model2/_mlp_optimizerADAM\checkpoint.ckpt and starting at epoch 51.
Loading optimizer state from model2/_mlp_optimizerADAM\checkpoint.optim and starting at epoch 51.
Loading random states from model2/_mlp_optimizerADAM\checkpoint.randomstate and starting at epoch 51.
Restoring data from model2/_mlp_optimizerADAM\checkpoint_epoch_9.ckpt


c:\Users\Utilisateur\anaconda3\Lib\site-packages\poutyne\framework\model_bundle.py:751: UserWarning: tensorboard does not seem to be installed. To remove this warning, set the 'disable_tensorboard' flag to True or install tensorboard.
  return self._train(self.model.fit_generator, train_generator, valid_generator, **kwargs)


In [20]:
experiment.test(test_dataset)

Found best checkpoint at epoch: 9
lr: 0.001, loss: 0.33516, acc: 89.3333, fscore_macro: 0.820262, val_loss: 1.14128, val_acc: 67.9849, val_fscore_macro: 0.476044
Loading checkpoint model2/_mlp_optimizerADAM\checkpoint_epoch_9.ckpt
Running test


TypeError: LSTMClassifier.forward() missing 1 required positional argument: 'x_lenght'

In [21]:
from poutyne.framework import Experiment
from poutyne import set_seeds
from torch.optim import SGD
import numpy as np

set_seeds(42)
hidden_size = 100
optimizer = "ADAM"

directory_name = 'model2/_mlp_optimizer{}/bidirectional'.format(optimizer)

bidirectional_experiment = Experiment(directory_name,
                        bidirectional_model,
                        optimizer = optimizer,
                        task="classification",
                        loss_function="cross_entropy",
                        )

In [22]:
logging = bidirectional_experiment.train(train_dataloader, valid_dataloader, epochs=50, disable_tensorboard=True)

Epoch:  1/50 Step:  34/155  21.94% |████▍               |ETA: 5m11.12s loss: 1.815479 acc: 37.500000

## 6. Évaluation et analyse de résultats